In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Loading Training Data
train_data = pd.read_csv("data/essay_data/training_set_rel3.tsv", delimiter="\t", encoding='ISO-8859-1')
train_data.head()

In [ ]:
# Selecting Relevant Columns
required_columns = ['essay_id', 'essay_set', 'essay', 'rater1_domain1', 'rater2_domain1', 'domain1_score']
train_data = train_data[required_columns]